<h1 style="background-color: DodgerBlue;color:white;padding:20px;text-align: center;"> <b>Regular expression trong Stata </b></h1>

In [1]:
// Thông tin về máy tính
di "Đào Huy Cừ"
di c(current_date)
di "Stata version: " c(stata_version) " SE"
di "Hệ điều hành: "  c(os) " " c(machine_type)
set varabbrev off


Đào Huy Cừ

22 Jul 2020

Stata version: 15.1 SE

Hệ điều hành: Windows PC (64-bit x86-64)



<h2> Regular expression trong stata được chia làm 3 cách sử dụng: <br></h2>
    <ul style="line-height: 180%">
 <li><b>regexm(s,re):</b> trả về giá trị 1 nếu thỏa mãn regular expression (re) trong s, nếu không trả về giá trị 0</li>
 <li><b>regexs(n):</b> lấy biểu thức con từ regexm() trước đó</li>
 <li><b>regexr(s1,re,s2):</b> Thay thế giá trị trong s1 thỏa mãn regular expression (re) bằng s2</li>
    </ul>

## 1. regexm():

In [2]:
qui {
    clear
    input str18 date
    20jan2007 
    16June06
    06sept1985
    21june04
    4july90
    9jan1999
    6aug99
    19august2003
    aug19-2003
    jan9-2003
    july4-2004
    end
}
list



. list

     +--------------+
     |         date |
     |--------------|
  1. |    20jan2007 |
  2. |     16June06 |
  3. |   06sept1985 |
  4. |     21june04 |
  5. |      4july90 |
     |--------------|
  6. |     9jan1999 |
  7. |       6aug99 |
  8. | 19august2003 |
  9. |   aug19-2003 |
 10. |    jan9-2003 |
     |--------------|
 11. |   july4-2004 |
     +--------------+


In [3]:
//Lọc danh sách những ngày bắt đầu từ con số
list date if regexm(date,"^[0-9]+")


     +--------------+
     |         date |
     |--------------|
  1. |    20jan2007 |
  2. |     16June06 |
  3. |   06sept1985 |
  4. |     21june04 |
  5. |      4july90 |
     |--------------|
  6. |     9jan1999 |
  7. |       6aug99 |
  8. | 19august2003 |
     +--------------+


In [4]:
//Lọc danh sách những ngày bắt đầu bằng tháng
list date if regexm(date,"^[a-zA-Z]")


     +------------+
     |       date |
     |------------|
  9. | aug19-2003 |
 10. |  jan9-2003 |
 11. | july4-2004 |
     +------------+


In [5]:
//Lọc danh sách những ngày kết thúc là năm có 2 chữ số
list date if regexm(date,"[a-z][0-9][0-9]$")


     +----------+
     |     date |
     |----------|
  2. | 16June06 |
  4. | 21june04 |
  5. |  4july90 |
  7. |   6aug99 |
     +----------+


Notes:
- "^": Để đánh dấu là bắt đầu của chuỗi string
- "$": Để đánh dấu là kết thúc của chuỗi string
- "+": thể hiện regular expression có thể được thỏa mãn 1 hoặc nhiều hơn 1 lần

## 2. regexs():

In [6]:
//Tạo biến ngày từ biến date đẫ cho (lấy từng giá trị mong muốn)
qui {
    cap drop day
    gen     day1 = regexs(0) if regexm(date,"(^[0-9]+)")
    replace day1 = regexs(1) if regexm(date,"^[a-zA-Z]+([0-9]+)[-]") & mi(day1)
    
    cap drop month
    gen month1 = regexs(1) if regexm(date,"^[0-9]+([a-zA-Z]+)")
    replace month1 = regexs(0) if regexm(date,"^[a-z]+") & mi(month1)
    
    cap drop year
    gen year1 = regexs(0) if regexm(date,"([0-9]+$)")
}
list



. list

     +--------------------------------------+
     |         date   day1   month1   year1 |
     |--------------------------------------|
  1. |    20jan2007     20      jan    2007 |
  2. |     16June06     16     June      06 |
  3. |   06sept1985     06     sept    1985 |
  4. |     21june04     21     june      04 |
  5. |      4july90      4     july      90 |
     |--------------------------------------|
  6. |     9jan1999      9      jan    1999 |
  7. |       6aug99      6      aug      99 |
  8. | 19august2003     19   august    2003 |
  9. |   aug19-2003     19      aug    2003 |
 10. |    jan9-2003      9      jan    2003 |
     |--------------------------------------|
 11. |   july4-2004      4     july    2004 |
     +--------------------------------------+


In [7]:
//Gộp các thông tin vào chung 1 expression
qui {
    local rege1 regexm(date,"(^[0-9]+)([a-zA-Z]+)([0-9]+$)")
    local rege2 regexm(date,"(^[a-zA-Z]+)([0-9]+)[-]([0-9]+$)")

    gen day2 = ""
    replace day2 = regexs(1) if `rege1'
    replace day2 = regexs(2) if `rege2'

    gen month2 = ""
    replace month2 = regexs(2) if `rege1'
    replace month2 = regexs(1) if `rege2'

    gen year2 = "" 
    replace year2 = regexs(3) if `rege1'
    replace year2 = regexs(3) if `rege2'
}
list, noobs



. list, noobs

  +--------------------------------------------------------------+
  |         date   day1   month1   year1   day2   month2   year2 |
  |--------------------------------------------------------------|
  |    20jan2007     20      jan    2007     20      jan    2007 |
  |     16June06     16     June      06     16     June      06 |
  |   06sept1985     06     sept    1985     06     sept    1985 |
  |     21june04     21     june      04     21     june      04 |
  |      4july90      4     july      90      4     july      90 |
  |--------------------------------------------------------------|
  |     9jan1999      9      jan    1999      9      jan    1999 |
  |       6aug99      6      aug      99      6      aug      99 |
  | 19august2003     19   august    2003     19   august    2003 |
  |   aug19-2003     19      aug    2003     19      aug    2003 |
  |    jan9-2003      9      jan    2003      9      jan    2003 |
  |------------------------------------------

## 3. regexr()

In [8]:
di date[10]
replace date = regexr(date,"^jan","juanuary") //Thay những ngày bắt đầu bằng jan thành january
di date[10]


jan9-2003

(1 real change made)

juanuary9-2003


In [9]:
list, noobs


  +----------------------------------------------------------------+
  |           date   day1   month1   year1   day2   month2   year2 |
  |----------------------------------------------------------------|
  |      20jan2007     20      jan    2007     20      jan    2007 |
  |       16June06     16     June      06     16     June      06 |
  |     06sept1985     06     sept    1985     06     sept    1985 |
  |       21june04     21     june      04     21     june      04 |
  |        4july90      4     july      90      4     july      90 |
  |----------------------------------------------------------------|
  |       9jan1999      9      jan    1999      9      jan    1999 |
  |         6aug99      6      aug      99      6      aug      99 |
  |   19august2003     19   august    2003     19   august    2003 |
  |     aug19-2003     19      aug    2003     19      aug    2003 |
  | juanuary9-2003      9      jan    2003      9      jan    2003 |
  |------------------------------